<a href="https://colab.research.google.com/github/kerimoglutolga/AdversarialLearning/blob/master/GATconditioned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np 

Not working conditioned GAT implementation

In [2]:
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

train_set = MNIST(
    './', 
    download=True,
    transform=transforms.ToTensor(), 
    train=True
)

test_set = MNIST(
    './', 
    download=True,
    transform=transforms.ToTensor(), 
    train=False
)

--2021-10-04 12:55:22--  http://www.di.ens.fr/~lelarge/MNIST.tar.gz
Resolving www.di.ens.fr (www.di.ens.fr)... 129.199.99.14
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.di.ens.fr/~lelarge/MNIST.tar.gz [following]
--2021-10-04 12:55:23--  https://www.di.ens.fr/~lelarge/MNIST.tar.gz
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘MNIST.tar.gz.10’

MNIST.tar.gz.10         [           <=>      ]  33.20M  6.33MB/s    in 5.8s    

2021-10-04 12:55:29 (5.76 MB/s) - ‘MNIST.tar.gz.10’ saved [34813078]

MNIST/
MNIST/raw/
MNIST/raw/train-labels-idx1-ubyte
MNIST/raw/t10k-labels-idx1-ubyte.gz
MNIST/raw/t10k-labels-idx1-ubyte
MNIST/raw/t10k-images-idx3-ubyte.gz
MNIST/raw/train-images-idx3-ubyte
MNIST/raw/train-labels-idx1-ubyte.gz
MNIST/raw/t10k-images-idx3-ubyte
MNIST/r

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=128, shuffle=False)

In [4]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.label_emb = nn.Embedding(10, 10)

       
        self.add_conditions = nn.Sequential(
        nn.Linear(28*28 + 10, 256), nn.LeakyReLU(0.2, inplace=True),
        nn.Linear(256, 784), nn.LeakyReLU(0.2, inplace=True),    
        )
        self.net = nn.Sequential(
        nn.Conv2d(1, 48, kernel_size=3, stride=1, padding=1), nn.LeakyReLU(0.2, inplace=True),
        nn.Conv2d(48, 48, kernel_size=3, stride=1, padding=1), nn.LeakyReLU(0.2, inplace=True),
        nn.Conv2d(48, 48, kernel_size=3, stride=1, padding=1), nn.ReLU(),
        nn.Conv2d(48, 48, kernel_size=3, stride=1, padding=1), nn.ReLU(),
        nn.Conv2d(48, 48, kernel_size=3, stride=1, padding=1), nn.ReLU(),
        nn.Conv2d(48, 48, kernel_size=3, stride=1, padding=1), nn.ReLU(),
        nn.Conv2d(48, 1, kernel_size=1), nn.Tanh()
        )

    def forward(self, x, labels):
        # Concatenate label embedding and image to produce input
        img = torch.cat((x.view(x.size(0), -1),   self.label_emb(labels)), -1)
        img = self.add_conditions(img)
        img = img.view(img.size(0),1, 28,28)
        img = self.net(img)
        return img


In [5]:
class Classifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.label_emb = nn.Embedding(10, 10)
    self.add_conditions = nn.Sequential(
      nn.Linear(28*28 + 10, 256), nn.LeakyReLU(0.2, inplace=True),
      nn.Linear(256, 784), nn.LeakyReLU(0.2, inplace=True),    
    )
    self.net = nn.Sequential(
        nn.Conv2d(1, 48, kernel_size=3), nn.ReLU(),
        nn.Conv2d(48, 48, kernel_size=3, stride=2, padding=1), nn.ReLU(),
        nn.Conv2d(48, 96, kernel_size=3), nn.ReLU(),
        nn.Conv2d(96, 96, kernel_size=3, stride=2, padding=1), nn.ReLU(),
        nn.Conv2d(96, 96, kernel_size=3, stride=1, padding=1), nn.ReLU(),
        nn.Conv2d(96, 96, kernel_size=1, stride=1, padding=1), nn.ReLU(),
        nn.Conv2d(96, 10, kernel_size=1), 
        nn.AvgPool2d(kernel_size=8),
    )
  def forward(self, x, labels):
    img = torch.cat((x.view(x.size(0), -1),   self.label_emb(labels)), -1)
    img = self.add_conditions(img)
    img = img.view(img.size(0), 1, 28,28)
    img = self.net(img)
    logits = F.softmax(img, dim=1)
    return logits.view(-1, 10)

In [6]:
epochs = 20
epsilon = 0.1
alpha = 0.5
cg = 0.5
k = 1

FloatTensor = torch.cuda.FloatTensor 
LongTensor = torch.cuda.LongTensor 

generator = Generator().cuda()
classifier = Classifier().cuda()
adversarial_loss = torch.nn.MSELoss()
# Standard (non-adversarial) training loop
def train(device, train_loader):
  generator_optimizer = torch.optim.Adam(generator.parameters(), lr=1e-6)
  classifier_optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-3)
  loss = nn.CrossEntropyLoss()
  for epoch in range(epochs):
    for i, (x,y) in enumerate(train_loader):
      x, y = x.to(device), y.to(device)

      valid = Variable(FloatTensor(y.size(0), 1).fill_(1.0), requires_grad=False)
      fake = Variable(FloatTensor(y.size(0), 1).fill_(0.0), requires_grad=False)


      gen_labels = Variable(LongTensor(np.random.randint(0, 10, y.size(0))))
      
    
      generator_optimizer.zero_grad()
      perturbation = generator(x, gen_labels)
      probs = F.softmax(classifier(x + perturbation, gen_labels), dim=1)
      batch_loss = probs + cg * torch.norm(perturbation, p=2)
      validity_loss = loss(probs, gen_labels)
      gen_loss = 0.5*batch_loss.mean() + 0.5*validity_loss
      gen_loss.backward()
      generator_optimizer.step()

      """try:
        x, y = next(train_loader_iter)
      except StopIteration:
        train_loader_iter = iter(train_loader)
        x, y = next(train_loader_iter)
      x,y = x.to(device), y.to(device)"""
      classifier_optimizer.zero_grad()
      perturbation = generator(x, gen_labels)
      classifier_loss = alpha * loss(classifier(x, gen_labels), y) + (1-alpha) * loss(classifier(x+perturbation, gen_labels), y) + loss(classifier(x+perturbation, gen_labels), gen_labels)
      classifier_loss.backward()
      classifier_optimizer.step()      
      if (i%100 == 0):
        print('Epoch [{}/{}], Step [{}/{}], Generator_loss: {}, Classifier_loss: {}'.format(epoch+1, epochs, i+1, len(train_loader), gen_loss, classifier_loss))


In [7]:
train(device, train_loader) 

Epoch [1/20], Step [1/469], Generator_loss: 2.565598487854004, Classifier_loss: 4.60506534576416
Epoch [1/20], Step [101/469], Generator_loss: 2.3501930236816406, Classifier_loss: 4.37032413482666
Epoch [1/20], Step [201/469], Generator_loss: 2.1378402709960938, Classifier_loss: 4.198477745056152
Epoch [1/20], Step [301/469], Generator_loss: 1.9095795154571533, Classifier_loss: 4.040802001953125
Epoch [1/20], Step [401/469], Generator_loss: 1.6887869834899902, Classifier_loss: 3.9816696643829346
Epoch [2/20], Step [1/469], Generator_loss: 1.549318552017212, Classifier_loss: 3.946845531463623
Epoch [2/20], Step [101/469], Generator_loss: 1.4010130167007446, Classifier_loss: 3.9434688091278076
Epoch [2/20], Step [201/469], Generator_loss: 1.3652894496917725, Classifier_loss: 3.9540655612945557
Epoch [2/20], Step [301/469], Generator_loss: 1.3508189916610718, Classifier_loss: 3.954996109008789
Epoch [2/20], Step [401/469], Generator_loss: 1.3372236490249634, Classifier_loss: 3.93001985549

In [11]:
total = 0
adv_correct = 0
for x, y in test_loader:
  x, y = x.to(device), y.to(device)
  gen_labels = Variable(LongTensor(np.random.randint(0, 10, y.size(0))))
  perturbation = generator(x, gen_labels)
  perturbed_image = x + perturbation

  adv_out = classifier(perturbed_image, gen_labels)
  _, adv_pred = torch.max(adv_out.data, 1)
  total += y.size(0)
  adv_correct += (adv_pred == y).sum().item()

print("Adversarially trained network's accuracy:", adv_correct/total)

Adversarially trained network's accuracy: 0.0892
